# Optim HW 2 : Gradient methods
___
Martin Guyard

**Due date: 20 Nov. 2019**
___

## Problem 1 
___

Goal is to minimize unconstrained $f$:

$$ f(x_1,x_2,...,x_m) = \sum_{i=1}^{m} a_1\cdot(x_i-b_i)^2 + 3 $$
 
$$ x_{i}^{k+1} = x_{i}^{k} - t\nabla{f(x_{i}^{k})} $$
$$ x_{i}^{k+1} = x_{i}^{k} - t\cdot2\cdot a_i(x_{i}^{k} - b_i) $$

___
**Analytical solution**:

$f$ is minimum when $\forall i, x_i = b_i$. Moreover we can solve this problem with gradient descent in one step with:
$\forall i, x_{i}^{k} - t\cdot2\cdot a_i(x_{i}^{k} - b_i) = b_i $, means that $\forall{i},  2\cdot t \cdot a_i = 1$.

**One trivial solution is $t=0.5$ and $\forall i, a_i=1$**
___
**Convergence analysis**:

The gradient descent algorithm converge on $t<1/L$ given that $f$ is convex and $L$ Lipschitz.

$f$ is $L$ Lipschitz if $\| \nabla f(x) - \nabla f(y)\|_2 \leq L\|x-y\|_2$


$\|A(X-B) - A(Y-b)\|_2 \leq L \|X-Y\|_2$

$\|AX - AY\|_2 \leq L \|X-Y\|_2$

$\|A(X-Y)\|_2 \leq L \|X-Y\|_2$

$max(A)\|X-Y\|_2 \leq L \|X-Y\|_2$

$L \geq max(A)$

**if $\forall i, a_i=1$, then the convergence criterium is $L \geq 1$ and $t \leq 1$**

Gradient (and gradient+backtracking) convergence rate is $O(1/k)$
___

### Gradient Descent 

In [1]:
from math import sqrt
from random import randrange
import numpy as np

def norm(K, L=None):
    # return euclidean distance between vector K and L
    if L:
        X = [k-l for k, l  in zip(K, L)]
    else:
        X = K
    return sqrt(sum(x**2 for x in X))

In [2]:
# Grad is a lambda function that compute the gradient of f
# update is a lambda function that compute the descent: x(k+1)

def gradient_descent(X, grad, update, epsilon=1e-6, step=0.5):
    gradf = grad
    descent = update
    
    X_k = list(X)
    X_k1 = descent(X_k, step)
    
    steps = 0
    while norm(X_k, X_k1) >= epsilon:
        X_k2 = descent(X_k1, step)
        X_k = list(X_k1)
        X_k1 = list(X_k2)
        steps += 1
        
    print("Gradient descent steps:", steps)
    print("[ step size t =", step, ', convergence criterium 1/t <', max(A),']\n')
    return X_k   

### Gradient Descent with Backtracking

In [79]:
def gradient_descent_bt(X, func, grad, update, epsilon=1e-6, step=0.5, alpha=0.5, beta=0.5, verbose=True):
    f = func
    gradf = grad
    descent = update

    X_k = list(X)
    X_k1 = descent(X_k, step)
    
    steps = 0
    while norm(X_k, X_k1) >= epsilon:
        while f(X_k1) > f(X_k) - alpha * step * norm(gradf(X_k)) ** 2:
            step *= beta
            X_k1 = descent(X_k, step)
        else:
            X_k2 = descent(X_k1, step)
            X_k = list(X_k1)
            X_k1 = list(X_k2)
        steps += 1
    
    if verbose:
        print('Gradient descent with backtracking steps:', steps)
        print('[ step size t =', step, ', alpha =', alpha, ', beta =', beta, ']\n')
    return X_k 

___
Assume that:

- $ m = 500 $

- $ \forall i, a_i = 1 $

- $ \forall i, b_i \sim U(0, 100) $

- $ \forall i, x_i = 0 $ for initial x

**As we already analyticaly solved this precise problem, I choose step size t = 1**

**The stopping condition is $d(x_{k+1}, x_{k}) \leq \varepsilon$ with default $\varepsilon = 1e^{-6}$ and $d(.)$ the euclidean distance**

**Note: The stopping condition could also be X == B or norm(f_grad(X)) <= epsilon**

In [93]:
m = 500
X = [0] * m
A = [1] * m
B = [randrange(0, 100) for i in range(m)]

**!!!The code is made to be reused!!!**

f, gradf and descent are python lambda function. These function are not harcoded into gradient_descent(...) and gradient_descent_with_backtracking(...). This makes easier code reuse for problem2.

**recall:** 

f: $f(x_1,x_2,...,x_m) = \sum_{i=1}^{m} a_i\cdot(x_i-b_i)^2 + 3$

gradf: $\nabla f(x_i) = 2a_i\cdot(x_i-b_i)$, **note:** that gradf returns a vector of $(\nabla f(x_1), ...,\nabla f(x_m))$

decent: $x_{i}^{k+1} = x_{i}^{k} - t\cdot2\cdot a_i(x_{i}^{k} - b_i)$, **note:** that descent returns a vector of $(x_1^{k+1}, ...,x_m^{k+1})$

In [94]:
f = lambda X: sum([(a*(x-b)**2) for x, a, b in zip(X, A, B)])+3 # compute f(X)
gradf = lambda X: [2*a*(x-b) for x, a, b in zip(X, A, B)] # compute the grad vector: ouput = list
descent = lambda X, t: [x-t*grdf for x, grdf in zip(X, gradf(X))] # compute the update vector: output = list

In [95]:
gd_res = gradient_descent(X, grad=gradf, update=descent, step=0.5)

# step size don't batter for backtracking because of adaptative step size
gd_bt_res = gradient_descent_bt(X, func=f, grad=gradf, update=descent, step=42)

# print(gd_res)
# print(gd_bt_res)

Gradient descent steps: 1
[ step size t = 0.5 , convergence criterium 1/t < 1 ]

Gradient descent with backtracking steps: 20
[ step size t = 0.328125 , alpha = 0.5 , beta = 0.5 ]



**The convergence speed is the same with step size t=0.5 since this solve the problem analytically as shown above.**

___
Assume that:

- $ m = 500 $

- $ \forall i, a_i \sim U(1, 100) $

- $ \forall i, b_i \sim U(1, 100) $

- $ \forall i, x_i = 0 $ for initial x

**I choose step size t=1/100 since max(A) < 100**

**The stopping condition is $d(x_{k+1}, x_{k}) \leq \varepsilon$ with default $\varepsilon = 1e^{-6}$ and $d(.)$ the euclidean distance**

**Note: The stopping condition could also norm(f_grad(X)) <= epsilon**

In [96]:
m = 500
X = [0] * m
A = [randrange(1, 100) for i in range(m)]
B = [randrange(1, 100) for i in range(m)]

In [97]:
gd_res = gradient_descent(X, grad=gradf, update=descent, step=1/100)

# step size don't batter for backtracking because of adaptative step size
gd_bt_res = gradient_descent_bt(X, func=f, grad=gradf, update=descent, step=42)

# print(gd_res)
# print(gd_bt_res)

Gradient descent steps: 944
[ step size t = 0.01 , convergence criterium 1/t < 99 ]

Gradient descent with backtracking steps: 1233
[ step size t = 0.005126953125 , alpha = 0.5 , beta = 0.5 ]



**The convergence speed is '*usually*' slower with backtracking as it is taking more steps, however backtracking advantage is not to converge faster, but to always converge, with batcktracking step size can be anything since the backtrack is updating the step size.**

## Problem 2
___


Goal is to **minimize** **constrained** $f$:

$$ f(x_1,x_2,...,x_m) = \sum_{i=1}^{m} a_1\cdot(x_i-b_i)^2 + 3 $$
$$\forall i, x_i \geq 0 $$
$$\sum_{i=0}^{m} x_i \leq 100 $$
<br>
<br>

#### Trying to find an anaytical solution

This problem has no equalities constraints, the Lagrangian is simply:
<br>$L(x, \lambda) = f(x) + \lambda h(x)$ where $x=(x_1, ... , x_m)$ and $\lambda=(\lambda_1, ..., \lambda_{m+1})$
<br>
$h_{m+1}(x)=(\sum_{i=1}^{m} x_i) -100 \leq 0$
<br>
$\forall i \in [1, m], h_{i}(x)=-x \leq 0$
<br>
<br>
<br>

**KKT conditions:**
- primal feasibility: $\forall i, -x_i \leq 0$ and $\sum_{i=1}^m x_i -100 \leq 0$
- dual feasibility: $\forall i \in [1, m+1], \lambda_i \geq 0$ and $\forall i \in [1,m], \lambda_i (-x_i) = 0$
- complementary slackness: $\forall i \in [1,m], x_i(2a_i(x_i-b_i)-\lambda_i + \lambda_{m+1})=0$ and $\lambda_{m+1} (\sum_{i=1}^{m} x_i - 100) = 0$


<br>
We consider $a_i, b_i \geq 1$
<br>
<br>
With complementary slackness, $xi=b_i+\frac{\lambda_i-\lambda_{m+1}}{2a_i}\geq0$ 

**Since $x_i \geq 0$ we got $x_i = max(0, b_i+\frac{\lambda_i-\lambda_{m+1}}{2a_i})$**

<br>


Then $\lambda_{m+1} (\sum_{i=1}^{m} x_i - 100) = 0$ implies that either $\lambda_{m+1} = 0$ or $\sum_{i=1}^{m} x_i - 100=0$
<br>
<br>
<br>

### DUAL ASCENT
___
$min f(x)$ constrained to
$\sum x_i -100 \leq 0$ and $x_i \geq 0$

**PRIMAL:** $min_x max_\lambda (f(x) +\lambda h(x))$

**DUAL:** $max_\lambda min_x (f(x) +\lambda h(x))$

<br>
<br>
dual ascent update:

$x^{k+1} = min_x(L(x^{k}, \lambda^{k}))$ **Note: we are going to use our previous implementation of gradient decent with backtracking to compute this**

$\lambda^{k+1}=\lambda^{k} - t(\sum x_i^{k+1} - 100)$, t is a fixed step in this example (choosen)

For the dual ascent to work, the first step is a minimization of the Lagrangian, we can use Gradient Descent Backtracking to ensure that we found a X*. 

In [182]:
m = 500
X = [0] * m
A = [randrange(1, 100) for i in range(m)]
B = [randrange(1, 100) for i in range(m)]
U = [0] * (m+1) # we have m+1 dual mutlipliers

In [183]:
# compute f
f = lambda X: sum([(a*(x-b)**2) for x, a, b in zip(X, A, B)])+3 # compute f(X)

# compute Lagrangian of f
lagrangian = lambda X: sum([(a*(x-b)**2) for x, a, b in zip(X, A, B)]) + 3 +\
             sum([(-x)*u for x, u in zip(X, U[:-1])]) + U[-1] * sum(X)

# compute for all i, grad(lagrangian(x_i))
gradL = lambda X: [2*a*(x-b)-u+U[-1] for x, a, b, u in zip(X, A, B, U[:-1])]

# update for all i, x_i(k+1) = x_i(k) - t*grad(lagrangian(x_i(k)))
descentL = lambda X, t: [x-t*grdf for x, grdf in zip(X, gradL(X))]

def descentU(U, X, t):
    U_k1 = list()
    for u, x in zip(U[:-1], X):
        U_k1.append(max(u+t*(u*x), 0))
    U_k1.append(max(U[-1]+t*(sum(X)-100), 0))
    return U_k1

gd_bt_lagrangian = gradient_descent_bt(X, func=lagrangian, grad=gradL, update=descentL)

Gradient descent with backtracking steps: 1728
[ step size t = 0.00390625 , alpha = 0.5 , beta = 0.5 ]



In [184]:
def dual_ascent(X, updateU, epsilon=1e-6, step=0.01):
    global U
    
    descentU = updateU
    
    X_k = list(X)
    X_k1 = gradient_descent_bt(X_k, func=lagrangian, grad=gradL, update=descentL,verbose=False)
    
    U_k = list(U)
    U_k1 = descentU(U_k, X_k1, step)
    U = list(U_k1)
     
    steps = 0
    while norm(X_k, X_k1) >= epsilon:
        X_k2 = gradient_descent_bt(X_k1, func=lagrangian, grad=gradL, update=descentL, verbose=False)
        U_k2 = descentU(U_k1, X_k2, step)
        
        X_k = list(X_k1)
        X_k1 = list(X_k2)
        
        U_k = list(U_k1)
        U_k1 = list(U_k2)
        U = list(U_k1) # IMPORTANT: update global variable
        
        steps += 1
        print("sum(X) = ", sum(X_k1), 'xi > 0 :', all(x >= 30 for x in X_k1))
    
    print('Dual ascent steps:', steps)
    print('[ sum(X*) =', sum(X_k1), ', xi > 0 :', all(x >= 30 for x in X_k1), ']\n')
    return X_k1
                                   
res = dual_ascent(X, updateU=descentU)
 

sum(X) =  21120.7646895289 xi > 0 : False
sum(X) =  18395.484273235914 xi > 0 : False
sum(X) =  16023.528688686356 xi > 0 : False
sum(X) =  13959.089970812107 xi > 0 : False
sum(X) =  12162.299671105657 xi > 0 : False
sum(X) =  10598.457900668873 xi > 0 : False
sum(X) =  9237.363605582195 xi > 0 : False
sum(X) =  8052.731461713046 xi > 0 : False
sum(X) =  7021.683109460639 xi > 0 : False
sum(X) =  6124.307187981797 xi > 0 : False
sum(X) =  5343.273389114252 xi > 0 : False
sum(X) =  4663.498401979053 xi > 0 : False
sum(X) =  4071.8542649649726 xi > 0 : False
sum(X) =  3556.9152758379723 xi > 0 : False
sum(X) =  3108.736509169715 xi > 0 : False
sum(X) =  2718.6627488067015 xi > 0 : False
sum(X) =  2379.1605204945986 xi > 0 : False
sum(X) =  2083.674173949245 xi > 0 : False
sum(X) =  1826.496615656044 xi > 0 : False
sum(X) =  1602.6614520111918 xi > 0 : False
sum(X) =  1407.8458192856915 xi > 0 : False
sum(X) =  1238.2874742504082 xi > 0 : False
sum(X) =  1090.7122596553106 xi > 0 : False

**Stopping condition**: same as gradient descent, when the distance between $x^{k}$ and $x^{k+1}$ is less than epsilon it means that the algorithm has converge to a solution.

**Step size**: I tried several, this one worked.

**Convergence rate**: O(1/k) k: number of steps

**The algorithm could be paralellize:** 

1thread compute the grad of L, 1thread compute the new dual multiplier, it would divide by a factor 2 the number of iteration to convergence.

<br>
<br>

___
Dual Ascent was really hard for me to understand, here are ressources I found online to help me better understand the principle, tho I am still not sure that I really get it.

**dual ascent YT tutorial: https://www.youtube.com/watch?v=HOx-fZ01VnY**

**exercise: https://bdesgraupes.pagesperso-orange.fr/UPX/Master1/MNM1_corr_doc2.pdf**

**Note:** on the code above U is a global variable that is use to compute the gradient of the lagrangian as well as the update of X. the global variable U is updated during the dual descent. ex below

In [31]:
G = 0

def foo():
    global G
    G = 10
    
print(G)
foo()
print(G)

0
10
